In [ ]:
%pip install trl==0.7.4 transformers==4.33.1 datasets==2.14.4 peft==0.5.0 ir-datasets llamaapi mistral_inference evaluate

  Using cached transformers-4.33.1-py3-none-any.whl.metadata (119 kB)
  Using cached datasets-2.14.4-py3-none-any.whl.metadata (19 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.33.1-py3-none-any.whl (7.6 MB)
Using cached datasets-2.14.4-py3-none-any.whl (519 kB)
Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
  Attempting uninstall: datasets
    Found existing installation: datasets 3.2.0
    Uninstalling datasets-3.2.0:
      Successfully uninstalled datasets-3.2.0
ERROR: pip's dependency resolver

In [ ]:
!pip install -U datasets
!pip install --upgrade transformers accelerate wandb

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
Using cached datasets-3.2.0-py3-none-any.whl (480 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.47.0-py3-none-any.whl (10.1 MB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.1
    Uninstalling transformers-4.33.1:
      Successfully uninstalled transformers-4.33.1


In [ ]:
from datasets import load_dataset
import pandas as pd
import torch
import torch.nn as nn
from datasets import Dataset
import tqdm
import wandb
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModel, DistilBertForSequenceClassification, DistilBertTokenizer,
from transformers import GPT2Model, GPT2LMHeadModel, AutoModelForCausalLM, AutoTokenizer

import transformers
from pathlib import Path

from google.colab import drive
drive.mount('/content/drive/')
AUX_DATA_ROOT = Path("/content/drive/MyDrive/Диплом")


import sys
sys.path.append(str(AUX_DATA_ROOT))

%load_ext autoreload
%autoreload 1

%aimport utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
PATH = '/content/drive/MyDrive/Диплом'

In [ ]:
docs = load_dataset('irds/cranfield', 'docs')
queries = load_dataset('irds/cranfield', 'queries')
qrels = load_dataset('irds/cranfield', 'qrels')



queries_pd = pd.DataFrame(queries)
docs_pd = pd.DataFrame(docs)
qrels_pd = pd.DataFrame(qrels)

label_map = {-1: 0, 0: 1, 1: 2, 2: 3, 3: 4, 4: 5}

In [ ]:
gpt_rank_data = []
bert_rank_data = []
# Формируем данные для ранжирования
for _, qrel in tqdm.tqdm(qrels_pd.iterrows(), total = len(qrels_pd)):
    query_id = qrel['query_id']
    doc_id = qrel['doc_id']
    relevance = qrel['relevance']

    # Найдем соответствующий текст запроса
    query_text = queries_pd[queries_pd['query_id'] == query_id]['text']
    if len(query_text) == 0:
      continue
    query_text = query_text.iloc[0]
    # Найдем текст документа
    doc_text = docs_pd[docs_pd['doc_id'] == doc_id]['title']
    if len(doc_text) == 0:
      continue

    doc_text = doc_text.iloc[0]
    prompt = f"Assess the relevance between the query = {query_text} and the document titled = {doc_text}. Assign a relevance score from 0 to 5, where 0 indicates no relevance and 5 indicates maximum relevance. Dont write anything except one number"
    gpt_rank_data.append({
        'text': prompt,
        'label': label_map[relevance]
    })
    bert_rank_data.append({
        'query': query_text,
        'title': doc_text,
        'label':  label_map[relevance],
    })

gpt_cranfield_ranking = Dataset.from_list(gpt_rank_data)

# Сохраняем созданный датасет (если нужно)



cranfield_ranking_num_classes = len(label_map)


bert_cranfield_ranking = Dataset.from_list(bert_rank_data)

100%|██████████| 1837/1837 [00:02<00:00, 745.63it/s]


Модели

In [ ]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# tokenizer.pad_token = tokenizer.eos_token

# model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=cranfield_ranking_num_classes, device_map = device)
# model.config.pad_token_id = model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=cranfield_ranking_num_classes)
# #bert_distil.classifier = nn.Sequential(nn.Linear(in_features=768, out_features=1), nn.Sigmoid())
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')




Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Датасеты

In [ ]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

class CranfieldPointwiseDatasetGpt(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.texts = dataset['text']
        self.label = dataset['label']

    def __len__(self):
        return len(self.texts) # all pairs

    def __getitem__(self, index: int):
        text = self.tokenizer(self.texts[index], truncation=True, padding='max_length', max_length=80)
        return dict(input_ids=text['input_ids'], attention_mask=text['attention_mask'], labels = self.label[index], )

gpt_cranfield_pointwise_dataset = CranfieldPointwiseDatasetGpt(gpt_cranfield_ranking, tokenizer)


In [ ]:
class CranfieldPointwiseDatasetBert(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.query = dataset['query']
        self.title = dataset['title']
        self.label = dataset['label']

    def __len__(self):
        return len(self.query) # all pairs

    def __getitem__(self, index: int):
      text_pair = self.tokenizer(
          self.query[index],
          self.title[index],
          truncation=True,
          padding='max_length',
          max_length=80,
          return_tensors='pt'
      )
      # Flatten the outputs to remove extra dimension introduced by `return_tensors='pt'`
      text_pair = {key: tensor.squeeze(0) for key, tensor in text_pair.items()}
      # Return as a dictionary
      return {
          'input_ids': text_pair['input_ids'],
          'attention_mask': text_pair['attention_mask'],
          'labels': self.label[index]
      }



bert_cranfield_pointwise_dataset = CranfieldPointwiseDatasetBert(bert_cranfield_ranking, tokenizer)


Запуск модели

In [ ]:
config = {
    'model': model,
    'tokenizer': tokenizer,
    'LLora': True,
    'PTune': False,
    'FineTune': False,
    'batch_size': 32,
    'num_epochs': 2,
}

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Bsc Diplom",
    reinit=True,
    # track hyperparameters and run metadata,
    config = config,
)

In [ ]:
trainer = utils.run_classifier(gpt_cranfield_pointwise_dataset,
    tokenizer = config['tokenizer'],
    model = config['model'],
    LLora = config['LLora'],
    batch_size = config['batch_size'],
    num_epochs = config['num_epochs'])

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
10,4.860500,4.654258,0.411444,0.239877,0.169286,0.411444
20,4.268000,4.370849,0.411444,0.239877,0.169286,0.411444
30,3.965000,4.047719,0.411444,0.239877,0.169286,0.411444
40,3.569900,3.709454,0.411444,0.239877,0.169286,0.411444
50,3.177300,3.387136,0.411444,0.239877,0.169286,0.411444


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
10,4.860500,4.654258,0.411444,0.239877,0.169286,0.411444
20,4.268000,4.370849,0.411444,0.239877,0.169286,0.411444
30,3.965000,4.047719,0.411444,0.239877,0.169286,0.411444
40,3.569900,3.709454,0.411444,0.239877,0.169286,0.411444
50,3.177300,3.387136,0.411444,0.239877,0.169286,0.411444
60,2.854500,3.109014,0.411444,0.239877,0.169286,0.411444
70,2.854800,2.895715,0.411444,0.239877,0.169286,0.411444
80,2.474300,2.760059,0.411444,0.239877,0.169286,0.411444
90,2.453500,2.704288,0.411444,0.239877,0.169286,0.411444


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Запуск LLM

In [ ]:
scores = run_LLM(gpt_cranfield_pointwise_dataset, model, tokenizer)